In [1]:
cd ..

/home/cody/inner-knowledge-pruning


In [2]:
from fastai.vision.all import *
from fastai.distributed import *
import torch
import torch.nn as nn
import torchvision
from torchvision import models
import torchvision.transforms as transforms

from src.dataloader import make_data_loader

In [3]:
dataloaders, _ = make_data_loader(32, 'imagewoof')

In [4]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7fae6c16e850>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x7fae6c169070>}

In [5]:


dls = DataLoaders(dataloaders['train'], dataloaders['val'])

In [6]:
model = models.vgg16_bn(pretrained=False)

In [7]:
model.classifier[-1] = nn.Linear(in_features=4096, out_features=10)
model = model.cuda()

In [8]:
learn = Learner(dls, model, loss_func=LabelSmoothingCrossEntropy(), metrics=accuracy, wd=0.01, cbs=[CudaCallback, SaveModelCallback()], path=Path.cwd()/'tmp_cifar10')
learn.model = learn.model.cuda()

In [9]:
learn.fit_one_cycle(240, 4e-4, wd=1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.277469,2.177852,0.200051,01:16
1,2.184194,2.042687,0.281751,01:14
2,2.146075,2.009547,0.320438,01:14
3,2.061072,1.936697,0.346399,01:14
4,2.009619,1.906137,0.359124,01:14
5,1.990816,1.925895,0.330618,01:14
6,1.943733,1.818802,0.402392,01:15
7,1.909829,1.801834,0.411301,01:14
8,1.880949,1.747063,0.439552,01:14
9,1.824082,1.718696,0.461441,01:14


Better model found at epoch 0 with valid_loss value: 2.177851676940918.
Better model found at epoch 1 with valid_loss value: 2.042686939239502.
Better model found at epoch 2 with valid_loss value: 2.0095467567443848.
Better model found at epoch 3 with valid_loss value: 1.936697006225586.
Better model found at epoch 4 with valid_loss value: 1.9061373472213745.
Better model found at epoch 6 with valid_loss value: 1.818801999092102.
Better model found at epoch 7 with valid_loss value: 1.801833987236023.
Better model found at epoch 8 with valid_loss value: 1.7470625638961792.
Better model found at epoch 9 with valid_loss value: 1.7186964750289917.
Better model found at epoch 12 with valid_loss value: 1.5973091125488281.
Better model found at epoch 14 with valid_loss value: 1.566501498222351.
Better model found at epoch 16 with valid_loss value: 1.4690388441085815.
Better model found at epoch 18 with valid_loss value: 1.4624671936035156.
Better model found at epoch 25 with valid_loss value:

In [10]:
learn.validate()

Better model found at epoch 0 with valid_loss value: 0.8859760761260986.


(#2) [0.8011856079101562,0.8859760761260986]

learn.fit_one_cycle(120, 1.5e-3, wd=1e-3) : [0.7092109322547913,0.9151592254638672]

learn.fit_one_cycle(120, 4e-4, wd=1e-3) : [0.6876890063285828,0.9276432991027832]

learn.fit_one_cycle(120, 1e-4, wd=1e-3): [0.761014461517334,0.9164330959320068]


In [12]:
learn.save(Path().cwd()/'./models/imagewoof/vgg16')

Path('/home/cody/inner-knowledge-pruning/models/imagewoof/vgg16.pth')